In [4]:
import os
import glob
import fnmatch

def get_java_files(directory_path):
    
    java_files = []


    for root, dirnames, filenames in os.walk(directory_path):
        for filename in fnmatch.filter(filenames, "*.java"):
            java_files.append(os.path.join(root, filename))

    return java_files
java_files = get_java_files('/Users/promachowdhury/Desktop/fast-projects/bug-localisation-backend/project/src')


In [39]:
import javalang
def extract_method_names(file_path):
    with open(file_path, 'r') as file:
        tree = javalang.parse.parse(file.read())

        method_names = []
        for path, node in tree:
            if isinstance(node, javalang.tree.MethodDeclaration):
                start_line = node.position.line
                method_names.append({'method_name':node.name, 'line':start_line})
                

    return method_names

method_names = {}

for file  in java_files:
    method_names[file] = extract_method_names(file)
    


In [10]:
method_names['/Users/promachowdhury/Desktop/fast-projects/bug-localisation-backend/project/src/main/java/org/jsoup/examples/HtmlToPlainText.java']

[{'method_name': 'main', 'line': 34},
 {'method_name': 'getPlainText', 'line': 61},
 {'method_name': 'head', 'line': 75},
 {'method_name': 'tail', 'line': 88},
 {'method_name': 'append', 'line': 97},
 {'method_name': 'toString', 'line': 126}]

In [16]:
import subprocess
import re
import concurrent.futures

    
def get_complexity(file_name):

    p = subprocess.Popen([f"java -jar /Users/promachowdhury/Desktop/fast-projects/bug-localisation-backend/lib/checkstyle-5.9-all.jar -c check.xml {file_name}"], shell=True,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output,stderr = p.communicate()
    output = output.decode('utf-8')


    pattern = r'(\S+\.java):(\d+):\d+: Cyclomatic Complexity is (\d+)'

    matches = re.finditer(pattern, output)

   
    file_info = []
    for match in matches:
        filename = match.group(1)
        line_number = match.group(2)
        complexity = match.group(3)
        file_info.append({"File": filename, "Line Number": line_number, "Cyclomatic Complexity": complexity})

    return file_info


complexity = []
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(get_complexity, file_path) for file_path in java_files]

    for future in concurrent.futures.as_completed(futures):
        try:
            result = future.result()
            complexity.extend(result)
        except Exception as e:
            complexity.extend({"Error": str(e)})





In [20]:
complexity[0].keys()

dict_keys(['File', 'Line Number', 'Cyclomatic Complexity'])

In [34]:
for res in complexity:
    meth_list = method_names[res['File']]
    methods = [m['method_name'] for m in meth_list]
    lines = [m['line'] for m in meth_list]

    method_line = next((i for i, r in enumerate(lines) if int(res['Line Number']) <= r), len(lines) - 1)
    if(method_line != -1):
        res['method_name'] = methods[method_line]



    



In [36]:
complexity[0]

{'File': '/Users/promachowdhury/Desktop/fast-projects/bug-localisation-backend/project/src/test/java/org/jsoup/nodes/LeafNodeTest.java',
 'Line Number': '12',
 'Cyclomatic Complexity': '1',
 'method_name': 'doesNotGetAttributesTooEasily'}

In [15]:
import javalang
from javalang.tree import MethodDeclaration


java_file_path = "/Users/promachowdhury/Desktop/fast-projects/bug-localisation-backend/project/src/main/java/org/jsoup/examples/HtmlToPlainText.java"


line_numbers_to_extract = [34, 61, 75, 88, 97, 125]

def extract_method_names_and_line_numbers(java_file_path, line_numbers_to_extract):
    with open(java_file_path, 'r') as java_file:
        source_code = java_file.read()

    tree_root = javalang.parse.parse(source_code)

    for path, node in tree_root.filter(MethodDeclaration):
        line_number = node.position.line
        
        method_name = node.name
        print(f"Line {line_number}: {method_name}")

if __name__ == "__main__":
    extract_method_names_and_line_numbers(java_file_path, line_numbers_to_extract)


Line 34: main
Line 61: getPlainText
Line 75: head
Line 88: tail
Line 97: append
Line 126: toString


In [14]:
import javalang
from javalang.tree import MethodDeclaration

# Define the path to the Java source code file
java_file_path = "/Users/promachowdhury/Desktop/fast-projects/bug-localisation-backend/project/src/main/java/org/jsoup/examples/HtmlToPlainText.java"

# Define the line number where you expect to find "override"
line_number_with_override = 125

# Create a function to extract method names and their line numbers
def extract_method_names_and_line_numbers(java_file_path, line_number_with_override):
    with open(java_file_path, 'r') as java_file:
        source_code = java_file.read()

    tree_root = javalang.parse.parse(source_code)

    method_names = {}
    method_started = False
    for path, node in tree_root:
        line_number = node.position.line
        if line_number == line_number_with_override and 'Override' in str(node.name): 
            method_started = True
        elif method_started and isinstance(node, MethodDeclaration):
            method_name = node.name
            method_names[line_number_with_override] = method_name
            method_started = False

    print(f"Line {line_number_with_override}: {method_names[line_number_with_override]}")

if __name__ == "__main__":
    extract_method_names_and_line_numbers(java_file_path, line_number_with_override)


AttributeError: 'NoneType' object has no attribute 'line'

In [13]:
import javalang
from javalang.tree import MethodDeclaration

# Define the path to the Java source code file
java_file_path = "/Users/promachowdhury/Desktop/fast-projects/bug-localisation-backend/project/src/main/java/org/jsoup/examples/HtmlToPlainText.java"

# Define the line number where you expect to find "override"
line_number_with_override = 125

# Create a function to extract method names and their line numbers
def extract_method_names_and_line_numbers(java_file_path, line_number_with_override):
    with open(java_file_path, 'r') as java_file:
        source_code = java_file.read()

    tree_root = javalang.parse.parse(source_code)

    method_names = {}
    method_started = False
    for path, node in tree_root:
        if hasattr(node, 'position') and node.position is not None:
            line_number = node.position.line
        else:
            line_number = None

        if line_number == line_number_with_override:
            
            if isinstance(node, javalang.tree.MethodDeclaration):
                method_name = node.name
                method_names[line_number_with_override] = method_name
                method_started = False
            elif 'Override' in str(node.name):  
                
                method_started = True

    if line_number_with_override in method_names:
        print(f"Line {line_number_with_override}: {method_names[line_number_with_override]}")

if __name__ == "__main__":
    extract_method_names_and_line_numbers(java_file_path, line_number_with_override)



yes


In [2]:
import os
import javalang

def find_test_methods(java_code):
    test_methods = []
    
    tree = javalang.parse.parse(java_code)
    
    for path, node in tree:
        if isinstance(node, javalang.tree.ClassDeclaration) and 'Test' in node.name:
            for method in node.methods:
                if any(annotation.name == 'Test' for annotation in method.annotations):
                    test_methods.append(method.name)
    
    return test_methods

def find_test_classes_and_methods(folder_path):
    test_classes_and_methods = {}
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".java"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as java_file:
                    java_code = java_file.read()
                    test_methods = find_test_methods(java_code)
                    if test_methods:
                        test_classes_and_methods[file] = test_methods
    
    return test_classes_and_methods

if __name__ == "__main__":
    folder_path = "/Users/promachowdhury/Desktop/fast-projects/bug-localisation-backend/project/src/test/"
    test_classes_and_methods = find_test_classes_and_methods(folder_path)
    
    for java_file, test_methods in test_classes_and_methods.items():
        print(f"Java file: {java_file}")
        for method in test_methods:
            print(f"  Test Method: {method}")


Java file: TextNodeTest.java
  Test Method: testBlank
  Test Method: testTextBean
  Test Method: testSplitText
  Test Method: testSplitAnEmbolden
  Test Method: testSplitTextValidation
  Test Method: testWithSupplementaryCharacter
  Test Method: testLeadNodesHaveNoChildren
  Test Method: testSpaceNormalise
  Test Method: testClone
  Test Method: testCloneAfterAttributesHit
  Test Method: testHasTextWhenIterating
  Test Method: createFromEncoded
  Test Method: normaliseWhitespace
  Test Method: stripLeadingWhitespace
  Test Method: leafNodeAttributes
Java file: LeafNodeTest.java
  Test Method: doesNotGetAttributesTooEasily
Java file: AttributesTest.java
  Test Method: html
  Test Method: testIteratorRemovable
  Test Method: testIteratorUpdateable
  Test Method: testIteratorHasNext
  Test Method: testIterator
  Test Method: testIteratorSkipsInternal
  Test Method: testListSkipsInternal
  Test Method: htmlSkipsInternals
  Test Method: testIteratorEmpty
  Test Method: testIteratorRemove
  

In [42]:
import getGitHubStats

he = getGitHubStats.aggregate_complexity()
he

TypeError: list indices must be integers or slices, not str